In [1]:
#pip install emoji

In [2]:
import numpy as np
import pandas as pd
import emoji
import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('/content/train_emoji.csv',header=None)
test = pd.read_csv('/content/test_emoji.csv',header=None)
#train = pd.read_csv('/content/train2_emoji.csv',header=None,encoding='ISO-8859-1',on_bad_lines='skip')
#test = pd.read_csv('/content/test2_emoji.csv',header=None,encoding='ISO-8859-1',on_bad_lines='skip')

In [4]:
#train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [5]:
#test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [6]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:", 5 : ":rage:"}

In [7]:
#for i in emoji_dict.keys():
  #print (i,end=" ")
  #print (emoji.emojize(emoji_dict[i], use_aliases=True))

0 ❤️
1 ⚾
2 😄
3 😞
4 🍴
5 😡


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The parameter 'use_aliases' in emoji.emojize() is deprecated and will be removed in version 2.0.0. Use language='alias' instead.
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#from sklearn.model_selection import train_test_split
#seed = 42

#X_train, X_test, Y_train, Y_test = train_test_split(train[0],train[1],test_size=0.33,random_state=seed)
#print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

In [9]:
# Creating training and testing data
X_train = train[0]
Y_train = train[1]
"""
Y_train = np.zeros((X_train.shape[0],1))
for i in range((X_train.shape[0])):
  for j in range(7):
    if (train[j+1][i]==1):
      Y_train[i]=j
"""
X_test = test[0]
Y_test = test[1]
"""
Y_test = np.zeros((X_test.shape[0],1))
for i in range((X_test.shape[0])):
  for j in range(7):
    if (test[j+1][i]==1):
      Y_test[i]=j
"""
#print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(141,) (141,) (56,) (56,)


In [10]:
for i in range(X_train.shape[0]):
  X_train[i] = X_train[i].split()

for j in range(X_test.shape[0]):
  X_test[j] = X_test[j].split()
    
Y_train = to_categorical(Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
embeddings_index = {}
f = open('/content/glove.6B.50d.txt', encoding="utf-8")
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

In [12]:
# stopwords
stopwords = []
stop = open('/content/stopwords.txt', encoding="utf-8")
for line in stop:
  stopwords.append(line.strip())
stop.close()

In [13]:
#print(stopwords)

In [14]:
#from scipy import spatial
# Checking cosine similarity of words happy and sad
#spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

In [15]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  6, 28, 37, 22, 21, 11,  6,  1,  5]))

In [16]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for i in range(X_train.shape[0]):
  for j in range(len(X_train[i])):
    if not (X_train[i][j].lower() in stopwords):
      if X_train[i][j].lower() in embeddings_index:
        embedding_matrix_train[i][j] = embeddings_index[X_train[i][j].lower()]
        
for i in range(X_test.shape[0]):
  for j in range(len(X_test[i])):
    if not (X_test[i][j].lower() in stopwords):
      if X_test[i][j].lower() in embeddings_index:
        embedding_matrix_test[i][j] = embeddings_index[X_test[i][j].lower()] 

In [17]:
embedding_out = np.zeros((1, 10, 50))
inputData = 'I was having my dinner'
input = inputData.split()
for i in range(len(input)):
  if not (input[i].lower() in stopwords):
    embedding_out[0][i] = embeddings_index[input[i].lower()]
    #print(input[i].lower())

In [18]:
#print (embedding_matrix_train.shape, embedding_matrix_test.shape)

In [19]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))

#model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
 activation (Activation)     (None, 6)                 0         
                                                                 
Total params: 224,006
Trainable params: 224,006
Non-trai

In [20]:
# Setting Loss ,Optimiser for model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Training model
hist = model.fit(embedding_matrix_train,
                 Y_train,
                 epochs = 50, 
                 batch_size = 40,
                 shuffle=True)

Epoch 1/50
4/4 [==============================] - 4s 36ms/step - loss: 1.7708 - accuracy: 0.2553
Epoch 2/50
4/4 [==============================] - 0s 33ms/step - loss: 1.6772 - accuracy: 0.4610
Epoch 3/50
4/4 [==============================] - 0s 32ms/step - loss: 1.5745 - accuracy: 0.4823
Epoch 4/50
4/4 [==============================] - 0s 37ms/step - loss: 1.4655 - accuracy: 0.4965
Epoch 5/50
4/4 [==============================] - 0s 35ms/step - loss: 1.2640 - accuracy: 0.6028
Epoch 6/50
4/4 [==============================] - 0s 33ms/step - loss: 1.0476 - accuracy: 0.6312
Epoch 7/50
4/4 [==============================] - 0s 38ms/step - loss: 0.8953 - accuracy: 0.6667
Epoch 8/50
4/4 [==============================] - 0s 34ms/step - loss: 0.8464 - accuracy: 0.7092
Epoch 9/50
4/4 [==============================] - 0s 33ms/step - loss: 0.7082 - accuracy: 0.7376
Epoch 10/50
4/4 [==============================] - 0s 35ms/step - loss: 0.7134 - accuracy: 0.7163
Epoch 11/50
4/4 [============

In [22]:
# Prediction of trained model
pred = np.argmax(model.predict(embedding_matrix_test), axis=-1)
# pred = (model.predict(embedding_matrix_test) > 0.5).astype("int32")

In [23]:
predOut = np.argmax(model.predict(embedding_out), axis=-1)

#******************** OUTPUT *****************************#
print (inputData,emoji.emojize(emoji_dict[predOut[0]], use_aliases=True))

I was having my dinner 🍴


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The parameter 'use_aliases' in emoji.emojize() is deprecated and will be removed in version 2.0.0. Use language='alias' instead.
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  


### ACCURACY

In [24]:
# Calculating accuracy / score  of the model
#float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.7321428571428571

In [28]:
# Printing the sentences with the predicted and the labelled emoji
"""for i in range(embedding_matrix_test.shape[0]):
  if pred[i] != Y_test[i]:
    print(i)
    print (test[0][i],end=" ")
    print (emoji.emojize(emoji_dict[pred[i]], use_aliases=True),end=" ")
    print (emoji.emojize(emoji_dict[Y_test[i]], use_aliases=True))
    """

'for i in range(embedding_matrix_test.shape[0]):\n  if pred[i] != Y_test[i]:\n    print(i)\n    print (test[0][i],end=" ")\n    print (emoji.emojize(emoji_dict[pred[i]], use_aliases=True),end=" ")\n    print (emoji.emojize(emoji_dict[Y_test[i]], use_aliases=True))\n    '

In [26]:
"""
epochs = 50
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")
plt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Acc")

plt.title("Loss and Accuracy plot")
plt.xlabel("Epoch")
plt.ylabel("Loss / Accuracy")
plt.legend(loc = "lower left")
plt.savefig("plot.jpg")
"""

'epochs = 50\nplt.style.use("ggplot")\nplt.figure()\nplt.plot(np.arange(0, epochs), hist.history["loss"], label = "Train Loss")\nplt.plot(np.arange(0, epochs), hist.history["accuracy"], label = "Train Acc")\n\nplt.title("Loss and Accuracy plot")\nplt.xlabel("Epoch")\nplt.ylabel("Loss / Accuracy")\nplt.legend(loc = "lower left")\nplt.savefig("plot.jpg")\n'

In [27]:
"""
import oauth2 as oauth
import urllib.request

# See assignment1.html instructions or README for how to get these credentials

api_key = "XQeBLBfD7vZHIJ7BEHP8C84Pv"
api_secret = "jCw4FY9IHaJhBo0ntLXP4Xi1fBPLuyq2y4A0zeBxsJ0mngJz4U"
access_token_key = "1436156721936875520-T1I5OjMAcYeZmXBolU9Lu8nFW6DOII"
access_token_secret = "ZKbWqiUiTV5zEYiSbOkMwA0mAOSpEFOE0Rr0J3k8b8rME"

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"


http_handler  = urllib.request.HTTPHandler(debuglevel=_debug)
https_handler = urllib.request.HTTPSHandler(debuglevel=_debug)

'''
Construct, sign, and open a twitter request
using the hard-coded credentials above.
'''
def twitterreq(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.request.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response

def fetchsamples():
  url = "https://stream.twitter.com/1/statuses/sample.json"
  parameters = []
  response = twitterreq(url, "GET", parameters)
  for line in response:
    print (line.strip())

if __name__ == '__main__':
  fetchsamples()
  """